In [ ]:
def wuw(audio):
    ############### ▼패키지 임포트▼ ###############
    import os
    import pathlib
    import librosa
    import matplotlib.pyplot as plt
    import numpy as np
    from numpy.random.mtrand import randint
    import seaborn as sns
    import tensorflow as tf
    import soundfile as sf
    import shutil
    from tensorflow.keras import layers
    from tensorflow.keras import models
    ############################## ▲패키지 임포트▲ ##############################
    
    # 저장한 모델을 불러옵니다
    saved_path = '/content/drive/MyDrive/Desktop_Based_AI_Secretary/강성현/model_saved/'
    model = models.load_model(saved_path + 'WUW_v3.h5')
    
    # final_test_audio_path = '/content/data/mini_speech_commands/stop' + '/'
    # final_test_audio_path = '/content/data/mini_speech_commands/jarvis' + '/'
    final_test_audio_path = '/content/drive/MyDrive/Desktop_Based_AI_Secretary/강성현/dataset/WUW dataset/'

    final_test_audio_list = sorted(os.listdir(final_test_audio_path))

    final_test_files = []
    for i in range(len(final_test_audio_list[0:12])):
        final_test_files.append(final_test_audio_path + final_test_audio_list[i])

    commands = np.array(['stop', 'go', 'yes', 'up', 'right', 'down', 'left', 'jarvis', 'no'])

    def get_spectrogram(waveform):
        input_len = 16000
        waveform = waveform[:input_len]
        zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)
        waveform = tf.cast(waveform, dtype=tf.float32)
        equal_length = tf.concat([waveform, zero_padding], 0)
        spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
        spectrogram = tf.abs(spectrogram)
        spectrogram = spectrogram[..., tf.newaxis]
        return spectrogram

    def get_spectrogram_and_label_id(audio, label):
        spectrogram = get_spectrogram(audio)
        label_id = tf.math.argmax(label == commands)
        return spectrogram, label_id

    def decode_audio(audio_binary):
        audio, _ = tf.audio.decode_wav(contents=audio_binary)
        return tf.squeeze(audio, axis=-1)

    def get_label(file_path):
        parts = tf.strings.split(input=file_path, sep= os.path.sep)
        return parts[-2]

    def get_waveform_and_label(file_path):
        label = get_label(file_path)
        audio_binary = tf.io.read_file(file_path)
        waveform = decode_audio(audio_binary)
        return waveform, label

    AUTOTUNE = tf.data.AUTOTUNE

    def preprocess_dataset(files):
        files_ds = tf.data.Dataset.from_tensor_slices(files)
        output_ds = files_ds.map(map_func= get_waveform_and_label, num_parallel_calls= AUTOTUNE)
        output_ds = output_ds.map(map_func=get_spectrogram_and_label_id, num_parallel_calls= AUTOTUNE)
        return output_ds

    final_test_files = tf.convert_to_tensor(final_test_files)
    final_test_files_ds = preprocess_dataset(final_test_files)

    final_test_audio = []

    for audio, label in final_test_files_ds:
        final_test_audio.append(audio.numpy())

    final_test_audio = np.array(final_test_audio)
    y_final_test = np.argmax(model.predict(final_test_audio), axis=1)

    if y_final_test.count('8') / len(y_final_test) > 0.8:
        return True